# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"linked_variables":{}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

a         b     g           c           d            e   
5   102.948486  0.581335  1234  103.529821  102.367151   207.059642  \
6   350.959987 -0.035568  1234  350.924418  350.995555   701.848836   
7   162.017645  0.214483  1234  162.232128  161.803162   324.464256   
8   195.528893 -0.584247  1234  194.944646  196.113140   389.889292   
9    95.466984 -0.841848  1234   94.625136   96.308832   189.250272   
10  947.230840 -0.400572  1234  946.830268  947.631411  1893.660537   
11    1.241367 -0.934657  1234    0.306710    2.176024     0.613420   
12  939.281799  0.813352  1234  940.095151  938.468448  1880.190302   
13  324.817347  0.547684  1234  325.365031  324.269662   650.730062   
14  906.533193 -0.323270  1234  906.209923  906.856463  1812.419846   

              f  
5    204.734302  
6    701.991110  
7    323.606324  
8    392.226281  
9    192.617663  
10  1895.262823  
11     4.352048  
12  1876.936896  
13   648.539325  
14  1813.712926

In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

objective_c  objective_d
5   -103.529821   102.367151
6   -350.924418   350.995555
7   -162.232128   161.803162
8   -194.944646   196.113140
9    -94.625136    96.308832
10  -946.830268   947.631411
11    -0.306710     2.176024
12  -940.095151   938.468448
13  -325.365031   324.269662
14  -906.209923   906.856463

In [13]:
# This is also available as a method
vocs.objective_data(data)

objective_c  objective_d
5   -103.529821   102.367151
6   -350.924418   350.995555
7   -162.232128   161.803162
8   -194.944646   196.113140
9    -94.625136    96.308832
10  -946.830268   947.631411
11    -0.306710     2.176024
12  -940.095151   938.468448
13  -325.365031   324.269662
14  -906.209923   906.856463

In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-1.03529821e+02,  1.02367151e+02],
       [-3.50924418e+02,  3.50995555e+02],
       [-1.62232128e+02,  1.61803162e+02],
       [-1.94944646e+02,  1.96113140e+02],
       [-9.46251358e+01,  9.63088317e+01],
       [-9.46830268e+02,  9.47631411e+02],
       [-3.06709982e-01,  2.17602415e+00],
       [-9.40095151e+02,  9.38468448e+02],
       [-3.25365031e+02,  3.24269662e+02],
       [-9.06209923e+02,  9.06856463e+02]])

In [15]:
vocs.constraint_data(data)

constraint_e  constraint_f
5     205.059642   -204.734302
6     699.848836   -701.991110
7     322.464256   -323.606324
8     387.889292   -392.226281
9     187.250272   -192.617663
10   1891.660537  -1895.262823
11     -1.386580     -4.352048
12   1878.190302  -1876.936896
13    648.730062   -648.539325
14   1810.419846  -1813.712926

In [16]:
vocs.feasibility_data(data)

feasible_e  feasible_f  feasible
5        False        True     False
6        False        True     False
7        False        True     False
8        False        True     False
9        False        True     False
10       False        True     False
11        True        True      True
12       False        True     False
13       False        True     False
14       False        True     False

# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

A
1  6.000000e+66
2  6.000000e+66
3  6.000000e+66

In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

objective_c  objective_d
9         -1.0          inf
3         -2.0          inf
4         -3.0          inf
5         -4.0          inf